# Deep MNIST

In [2]:
import tensorflow as tf
import numpy as np

In [1]:
# Load MNIST Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Weight Initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(inital)

### Convolution and Pooling

https://www.tensorflow.org/api_guides/python/nn#Convolution


* tf.nn.conv2d(input, filter, strides, padding)

https://stackoverflow.com/questions/34619177/what-does-tf-nn-conv2d-do-in-tensorflow

* tf.nn,max_pool(input, filterSize, strides, padding)

https://stackoverflow.com/questions/37674306/what-is-the-difference-between-same-and-valid-padding-in-tf-nn-max-pool-of-t

ksize: https://stackoverflow.com/questions/38601452/the-usages-of-ksize-in-tf-nn-max-pool


In [ ]:
# Convolution and Pooling
"""
x: input [_, height, width, numOfChannels]
W: filter [fHeight, fWidth, numOfChannels, numOfFilters]

output: [_, newHeight, newWidth, newChannels]
where newHeight = floor((height + 2p - fHeight) / stride) + 1      # general case
      newHeight = height                                           # same padding
and   newChannels = numOfFileters
"""
def conv(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')

In [ ]:
def cnn(x):
    """
    Build the convolutional neural network
    
    Args:
        x: input [m, 784], where 28*28=784 is the number of pixels in a MNIST image
    Returns:
        y: with the value equals to the logits which is going to be used in softmax
        keep_prob: a scalar placeholder for the probability of dropout, for further backpropogation
    """
    
    # Reshape x to use in CNN
    with tf.name_scope('reshape'):
        x_image = tf.reshape(x, [-1, 28, 28, 1]) # first dimension trick: -1 (when corresponding dimension of x is unknown yet) 
                                                # last dimension 1 for grayscale, or 3 for RGB
    
    # 1st Conv layer
    # 32 filters with size 5*5, number of channel is 1 (grayscale image)
    with tf.name_scope('conv1'):
        W_conv1 = weight_variable([5, 5, 1, 32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv(x_image, W_conv1) + b_conv1)
        
    # 1st pooling layer
    # the output will be []
    with tf.name_scope('pool1'):
        h_pool1 = max_pool(h_conv1)
    
    # 2nd Conv layer
    # 64 filters with size 5*5, number of channel should match the output of 1st Conv layer
    with tf.name_scope('conv2'):
        W_conv2 = weight_variable([5, 5, 32, 64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv(h_pool1, W_conv2) + b_conv2)
    
    # 2nd pooling layer
    with tf.name_scope('pool2'):
        h_pool2 = max_pool(h_conv2)
        
    # 1st FC layer
    # after 2 round of downsampling, the image height & width is 7 * 7
    with tf.name_scope('fc1'):
        W_fc1 = weight_variable([7*7*64, 1024])
        b_fc1 = bias_variable([1024])
        h_pool2_flat = tf.reshap(h_pool2, [-1, 7*7*64]) # the trick to flatten a matrix: -1
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
        # dropout regularization
        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    with tf.name_scope('fc2'):
        W_fc2 = weight_variable([1024, 10])
        b_fc2 = bias_variable([1])
        
        # before softmax
        y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
    return y_conv, keep_prob

### A trick to flatten a matrix X

e.g. shape of X: [a,b,c,d]
need to reshape to a matrix X_flatten of shape [b*c*d, a]

X_flatten = X.reshape(X.shape[0], -1).T

In [ ]:
def main():
    X = tf.placeholder(tf.float32, [None, 784])
    Y = tf.placeholder(tf.float32, [None, 10])
    
    # Build the architecture
    y_conv, keep_prob = cnn(X)
    
    # Compute the cost
    cross_entropy = tf.reduce.mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=y_conv))
    
    
    